In [1]:
import wandb
import tensorflow as tf
from wandb.keras import WandbCallback

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Input, Flatten, LSTM, GRU, Dense, Reshape, TimeDistributed, Bidirectional, LeakyReLU
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import AUC
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.layers import Dropout
from __datagen__ import DataGenerator
from keras import Sequential

from keras.utils import to_categorical

from types import SimpleNamespace
import keras
import numpy as np
import pandas as pd

import os 
from glob import glob
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
tf.config.run_functions_eagerly(True)

In [2]:
# Get list of physical devices
physical_devices = tf.config.experimental.list_physical_devices()
# Check if GPU is availab
gpu_devices = [device for device in physical_devices if device.device_type == 'GPU']

# Print whether GPU is available
if len(gpu_devices) > 0:
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [3]:
wandb.login(anonymous="allow")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jose-lunamr. Use `wandb login --relogin` to force relogin


True

BUILD MODEL

In [4]:
def conv_block(input, num_filters):

	x = Conv2D(num_filters, 3, padding="same")(input)
	x = BatchNormalization()(x)
	x = Activation("relu")(x)
	x = Conv2D(num_filters, 3, padding="same")(x)
	x = BatchNormalization()(x)
	x = Activation("relu")(x)
	return x 

def encoder_block(input, num_filters):
	
	x = conv_block(input, num_filters)
	p = MaxPool2D((2, 2))(x)
	return x, p

def dense_block(input_shape, num_classes):

	inputs = Input(shape=input_shape)
	
	x = Flatten()(inputs)
	dense_layer_1 = Dense(units=512, activation='relu')(x)
	dense_layer_1 = Dropout(0.4)(dense_layer_1)

	#dense_layer_2 = Dense(units=256, activation='relu')(dense_layer_1)
	#dense_layer_2 = Dropout(0.4)(dense_layer_2)
	output_layer = Dense(units=num_classes, activation='softmax')(dense_layer_1)

	return output_layer

def build_unet_encoder_model_lstm(input_shape):
    units = 128
    inputs = Input(input_shape)

	#ENCODER
    s1, p1 = encoder_block(inputs, units/2)
    s2, p2 = encoder_block(p1, units)
    s3, p3 = encoder_block(p2, units*2)
    s4, p4 = encoder_block(p3, units*4)
    b1 = conv_block(p4, units*8)
    b2 = Flatten()(b1)
    encoder_output = b2 
    
    return Model(inputs, encoder_output)

import keras.backend as K

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def LSTM_model(input_shape, units_f1layer):
	input_layer = Input(input_shape)
	lstm_layer1 = LSTM(units_f1layer, return_sequences=True)(input_layer)
	lstm_layer2 = LSTM(1024)(lstm_layer1)
	dense_layer1 = Dense(1024, activation="relu")(lstm_layer2)
	dense_layer2 = Dense(1024, activation="relu")(dense_layer1)
	last_lstm_layer = dense_layer2
	output_layer = Dense(13, activation="softmax")(last_lstm_layer)
	model = Model(inputs=input_layer, outputs=output_layer)
	return model

def LSTM_model_v1(input_shape, units_f1layer):
	model = Sequential()
									# model.add(LSTM(512, return_sequences=True, input_shape=(64, 32768)))
	model.add(LSTM(8000, return_sequences=True, input_shape=(64, 32768)))
	model.add(LSTM(128, activation="relu"))
	model.add(Dense(1024, activation="relu"))
	model.add(Dropout(0.2))
	model.add(Dense(512, activation="relu"))
									# model.add(BatchNormalization())
	model.add(Dense(128, activation="relu"))
	model.add(Dense(13, activation="softmax"))
	return model

def LSTM_model_v2(input_shape, units_f1layer):
	model = Sequential()
	model.add(Bidirectional(LSTM(256, return_sequences=True), input_shape=input_shape))
	model.add(Bidirectional(LSTM(128, activation="relu")))
	model.add(Dense(512, activation="relu", kernel_initializer="glorot_uniform", kernel_regularizer=tf.keras.regularizers.L2(1e-4)))
	model.add(Dense(256, activation="relu", kernel_initializer="glorot_uniform"))  # Regularización L2
	model.add(BatchNormalization())
	model.add(Dropout(0.2))
	model.add(Dense(13, activation="softmax",kernel_regularizer=tf.keras.regularizers.L2(1e-4)))
	return model




In [5]:
#compilar LSTM 64 datapoints, 32 mil caracteristicas

#Organize hyperparameters to track down
config = SimpleNamespace(
    input_shape = (64, 624),       # (64, 32768), Without 0s index drop
    LSTM_layer1 = 256, #number of images per study
    second_layer = 128,
    L_R = 23e-4,
    LOSS = "binary_crossentropy",
    METRICS = ["accuracy", AUC()],
    EPOCHS = 150,
    BATCH_SIZE = 32, 
)
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Crear el Callback para reducir la tasa de aprendizaje en función de la pérdida en validación
lr_scheduler = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=1e-6)


#class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)

CLASS_WEIGHTS = {
    0:  0.5103795,
    1: 24.5859375,
    2: 0.53393281,
    3: 7.8675,
    4: 0.5306914,
    5: 13.6826087,
    6: 23.48507463,
    7: 0.55679406,
    8: 6.07528958,
    9: 25.37903226,
    10: 0.56337272,
    11: 7.90703518,
    12: 10.1516129
}

#Start the wandb run
wandb.init(project="LSTM_normalized_Data_Bidirectional_ZeroDrop", config=config)

OPTIMIZER=tf.keras.optimizers.Adam(learning_rate=config.L_R)
main_model = LSTM_model_v2(config.input_shape, config.LSTM_layer1)
main_model.compile(optimizer=OPTIMIZER, loss=config.LOSS, metrics=config.METRICS)
main_model.summary()
tf.keras.utils.plot_model(main_model, to_file="/model.png", show_shapes = True )

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 64, 512)          1804288   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              656384    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 512)               131584    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                        

In [6]:
if __name__ == "__main__":

    checkpoint_filepath = 'D:/Downloads/rsna-2023-abdominal-trauma-detection/Experiments_ckpt/LSTM_V1.h5'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath,
                                                              monitor="loss",
                                                              save_best_only=True,
                                                              save_weights_only=False,
                                                              mode="min",
                                                              verbose=1)
    callbacks = [model_checkpoint_callback]
    data = pd.read_csv("C:/Users/Daniel/Desktop/RSNA_Abdominal_Trauma/local_database/train_data_lstm.csv")
    datagen = DataGenerator(data['Patient_id'], data['Series_id'], config.BATCH_SIZE)
    history = main_model.fit(datagen, epochs=config.EPOCHS, shuffle=False, callbacks=[callbacks, lr_scheduler,WandbCallback()], class_weight = CLASS_WEIGHTS)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
c:\Users\Daniel\miniconda3\envs\new_tf\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/150
148/148 [==============================] - ETA: 0s - loss: 0.5615 - accuracy: 0.0191 - auc: 0.8440
Epoch 1: loss improved from inf to 0.56153, saving model to D:/Downloads/rsna-2023-abdominal-trauma-detection/Experiments_ckpt\LSTM_V1.h5
148/148 [==============================] - 215s 1s/step - loss: 0.5615 - accuracy: 0.0191 - auc: 0.8440 - lr: 0.0023
Epoch 2/150
148/148 [==============================] - ETA: 0s - loss: 0.4248 - accuracy: 0.0000e+00 - auc: 0.9240
Epoch 2: loss improved from 0.56153 to 0.42477, saving model to D:/Downloads/rsna-2023-abdominal-trauma-detection/Experiments_ckpt\LSTM_V1.h5
148/148 [==============================] - 241s 2s/step - loss: 0.4248 - accuracy: 0.0000e+00 - auc: 0.9240 - lr: 0.0023
Epoch 3/150
148/148 [==============================] - ETA: 0s - loss: 0.4237 - accuracy: 0.0000e+00 - auc: 0.9234
Epoch 3: loss improved from 0.42477 to 0.42368, saving model to D:/Downloads/rsna-2023-abdominal-trauma-detection/Experiments_ckpt\LSTM_V1.h5